[This notebook is obsolete, it is only use for test ]

# Classifiers based on features extracted from matlab

In this notebook we use the different arff files obtained from matlab. We will use this features to obtain classifiers and test them in a cross validation process.

## A bit of set up

We need numpy and pandas for data. Pickle and gzip for read the extracted features

In [1]:
# set up Python environment: numpy for numerical routines
import numpy as np
import pandas as pd

# for store the results
import pickle
import gzip

import sys
sys.path.append('../pycode/')
import utilsData

In this example we only use the default linear SVM classifier from libsvm and the Gaussian NB from sklearn

In [2]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

## Feature reading

In this example only one package is read, but each ones have a size of 80Mb approximately.

In [3]:
data = pickle.load(gzip.open('../packages/AVA_info.pklz','rb',2))
num_images,num_info_vars = data.shape

In [4]:
total_files=['PHOG/','CHIST.arff', 'GHIST.arff', 'GIST_ori8_block4.arff', 'Centrist.arff']
phog_files=['2_bins360_levels0_angle360.arff', '3_bins300_levels0_angle360.arff', '4_bins200_levels0_angle360.arff',
            '5_bins100_levels0_angle360.arff', '6_bins50_levels0_angle360.arff', '7_bins20_levels0_angle360.arff',
            '8_bins100_levels1_angle360.arff', '9_bins50_levels1_angle360.arff', '10_bins20_levels1_angle360_redux.arff',
            '11_bins50_levels2_angle360.arff', '12_bins20_levels2_angle360.arff']

In [ ]:
if sys.argv[1] == '0':
    features = utilsData.readARFF('features/AVA/'+total_files[int(sys.argv[1])]+phog_files[int(sys.argv[2])])
else:
    features = utilsData.readARFF('features/AVA/'+total_files[int(sys.argv[1])])

In [5]:
# This block must be eliminated, is only for test in the notebook
batches = 100
delta = 2.5
sys.path.append('../pycode/')
import utilsData
features = utilsData.readARFF('../features/AVA/GHIST.arff')

In [6]:
num_features = len(features.columns)-1
data=pd.merge(data, features, on='id')
del features

In [7]:
data.loc[:,'id'] = data['id'].apply(str)
classes = np.array(data.sort_values(['id']).loc[:,'Class'])
features = np.array(data.sort_values(['id']).iloc[:,num_info_vars:num_features+num_info_vars])
means = np.array(data.sort_values(['id']).loc[:,'VotesMean'])

In [ ]:
# This block is for transfer the data for the C# implementation
data_for_c = data.sort_values(['id']).iloc[:,num_info_vars:num_features+num_info_vars]
data_for_c['Class'] = classes
data_for_c.to_csv('../packages/data_for_c.csv',index=False)

In [8]:
def balance_class(features, classes):
    classes_uniques = np.unique(classes)
    min_class = np.array([0,float('Inf')])
    for i in classes_uniques:
        aux_value = np.sum(classes == i)
        if aux_value < min_class[1]:
            min_class = np.array([i,aux_value])
            
    final_indexes = np.where(classes == min_class[0])[0]
    for i in classes_uniques:
        if i != min_class[0]:
            aux_indexes = np.where(classes == i)[0]
            #print np.random.choice(aux_indexes,replace=False,size=min_class[1])
            final_indexes = np.concatenate((final_indexes,np.random.choice(aux_indexes,replace=False,size=min_class[1])))
            
    final_indexes = np.sort(final_indexes)
    return (features[final_indexes],classes[final_indexes])
                                

In [9]:
train_features,train_classes = balance_class(features,classes)

In [ ]:
data_balanced_c = pd.DataFrame(train_features)
data_balanced_c['Class'] = train_classes
data_balanced_c.to_csv('../packages/data_balanced_c.csv',index=False)

In [ ]:
nbg_clf = GaussianNB()
nbg_clf.fit(features, classes)

In [10]:
nbg_clf = GaussianNB()
nbg_clf.fit(train_features, train_classes)

GaussianNB(priors=None)

In [ ]:
nbg_clf.predict_proba(data[data['id']=='10002'].iloc[:,num_info_vars:num_features+num_info_vars])

In [ ]:
predictions = nbg_clf.predict(features)

In [ ]:
np.sum(predictions==classes)/len(classes)

In [ ]:
data.sort_values(['id']).loc[np.logical_and(predictions==0,classes==0),'id']

In [11]:
import cv2

In [48]:
img = cv2.imread('20170829185812.png',0)
[counts,bins] = np.histogram(img,bins=256,range=[0,256])
gray_hist = counts/np.sum(counts)

In [37]:
nbg_clf.predict_proba(gray_hist)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.94989437,  0.05010563]])

In [38]:
nbg_clf.predict(gray_hist)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([0])

In [ ]:
indexes = np.array(range(0,len(classes))[:len(classes)-(len(classes) % batches)])
indexes = indexes.reshape((batches,-1))

## Cross validation

In this case, we prepare vectors with the batches of each fold in order to test them in galgo and store the results.

* First, we split the batches in 5 folds:

In [ ]:
np.random.seed(1000)
num_folds = 5
batches = 100
delta = 2
folds = np.random.choice(range(0,batches),replace=False,size=(num_folds,int(batches/num_folds)))

In [ ]:
sum_folds_svm = 0
sum_folds_nbg = 0
matrix_svm = np.zeros((2,2))
matrix_nbg = np.zeros((2,2))
for i in range(0, num_folds):
    
    # Prepare train
    train_indices = indexes[np.delete(folds,i,axis=0).reshape(-1)].reshape(-1)
    train_features = features[train_indices]
    train_classes = classes[train_indices]
    train_means = means[train_indices]
    
    # Delete values depending on the delta
    vector_out_delta = (train_means <= 5-delta) | (train_means >= 5+delta)
    train_features = train_features[vector_out_delta]
    train_classes = train_classes[vector_out_delta]
    
    # Class balance
    train_features,train_classes = balance_class(train_features,train_classes)
    
    # Fit models
    svm_clf = svm.LinearSVC()
    svm_clf.fit(train_features, train_classes)

    nbg_clf = GaussianNB()
    nbg_clf.fit(train_features, train_classes)
    
    # Prepare test
    test_indices = indexes[folds[i]].reshape(-1)
    test_features = features[test_indices]
    test_classes = classes[test_indices]
    
    # Evaluate SVM model
    predictions = svm_clf.predict(test_features)
    results = np.sum(predictions == test_classes)/float(len(predictions))
    sum_folds_svm += results
    
    matrix_svm[0,0] += np.sum(predictions[predictions == test_classes] == 0)
    matrix_svm[0,1] += np.sum(predictions[predictions != test_classes] == 1)
    matrix_svm[1,0] += np.sum(predictions[predictions != test_classes] == 0)
    matrix_svm[1,1] += np.sum(predictions[predictions == test_classes] == 1)
    
    # Evaluate gnb model
    predictions = nbg_clf.predict(test_features)
    results = np.sum(predictions == test_classes)/float(len(predictions))
    sum_folds_nbg += results
    
    matrix_nbg[0,0] += np.sum(predictions[predictions == test_classes] == 0)
    matrix_nbg[0,1] += np.sum(predictions[predictions != test_classes] == 1)
    matrix_nbg[1,0] += np.sum(predictions[predictions != test_classes] == 0)
    matrix_nbg[1,1] += np.sum(predictions[predictions == test_classes] == 1)

In [ ]:
data_results = {'accuracy':sum_folds_svm/num_folds, 'conf_matrix':matrix_svm, 'classifier':'SVM-L', 'descriptor':total_files[int(sys.argv[1])], 'delta':delta}
if sys.argv[1] == '0':
    data_results['case']=phog_files[int(sys.argv[2])]
    pickle.dump(data_results, gzip.open("results/SVM_balanced_Descriptor%d_Case%d.pklz" % (int(sys.argv[1]),int(sys.argv[2])), "wb" ), 2)
else:
    pickle.dump(data_results, gzip.open("results/SVM_balanced_Descriptor%d.pklz" % (int(sys.argv[1])), "wb" ), 2)

data_results = {'accuracy':sum_folds_nbg/num_folds, 'conf_matrix':matrix_nbg, 'classifier':'NB-G', 'descriptor':total_files[int(sys.argv[1])], 'delta':delta}
if sys.argv[1] == '0':
    data_results['case']=phog_files[int(sys.argv[2])]
    pickle.dump(data_results, gzip.open("results/GNB_balanced_Descriptor%d_Case%d.pklz" % (int(sys.argv[1]),int(sys.argv[2])), "wb" ), 2)
else:
    pickle.dump(data_results, gzip.open("results/GNB_balanced_Descriptor%d.pklz" % (int(sys.argv[1])), "wb" ), 2)


In [ ]:
dict_rest = pickle.load(gzip.open('../results/GNB_balanced_Descriptor0_Case0.pklz','rb',2))